# Ideas
- Align `latex` picture and text using `raisebox`
- For phi and psi and omega, build the angle with a glycine on either side, this would also require specifiying which residue the atoms belong to. LUckily, this information is in the perl script already.
- Call `latex` automatically with `rubber --clean foo.tex`.
- Use `openeye` python toolkit to render these angles, mayb.e

In [144]:
import pandas as pd
import subprocess as sp
import os as os

In [160]:
dihedrals = pd.DataFrame()

In [161]:
dihedrals = pd.read_csv('dihedlist.dat', sep='\s+', header=None, names=['Res', 'Angle', '1', '2', '3', '4'])

In [162]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [165]:
def write_input(atom_one, atom_two, atom_three, atom_four, residue, label):
    template ='''
from BuildStructure import placePeptide
from chimera import runCommand as rc
aa = "{0}"

# Build a sequence
placePeptide(aa, [(-50, 70)] * len(aa), model="peptide")
# Add hydrogens automatically
rc("addh")
# Align so the CA-CB bond is going into the page
rc("align @CA @N")
# Color and highlight
rc("color grey")
rc("repr stick")
rc("shape sphere radius 0.5 center @{1} color red")
rc("shape sphere radius 0.5 center @{2} color blue")
rc("shape sphere radius 0.5 center @{3} color green")
rc("shape sphere radius 0.5 center @{4} color orange")
# Save
rc("copy file {5}.png supersample 3")
# rc("close all")
'''.format(d[residue], atom_one, atom_two, atom_three, atom_four, label)
    return template

In [166]:
for angle in range(len(dihedrals)):
    atom_one = dihedrals['1'][angle]
    atom_two = dihedrals['2'][angle]
    atom_three = dihedrals['3'][angle]
    atom_four = dihedrals['4'][angle]
    residue = dihedrals['Res'][angle]
    label = residue + '-' + dihedrals['Angle'][angle]
    string = write_input(atom_one, atom_two, atom_three, atom_four, residue, label)
    os.chdir('render-dihedrals/') 
    chimera_file = open("tmp.py", "w")
    chimera_file.write(string)
    chimera_file.close()
    render('tmp.py')
    os.chdir('../')

In [109]:
def render(file):
    sp.call('chimera-headless --script {}'.format(file), shell=True)

In [142]:
def print_latex(residue, angle, atom_one, atom_two, atom_three, atom_four, figure):
    if angle == 'chi1':
        angle = 'chi_1'
    elif angle == 'chi2':
        angle = 'chi_2'
    elif angle == 'chi3':
        angle = 'chi_3'
    elif angle == 'chi4':
        angle = 'chi_4'
    elif angle == 'chi5':
        angle = 'chi_5'
    print('{} & $\{}$ & {} & {} & {} & {} & \includegraphics[height=1in]{{{}.png}} \\\\'.format(residue, angle,
                                                                             atom_one, atom_two, atom_three, atom_four, figure))

In [143]:
for row in range(len(dihedrals)):
    print_latex(dihedrals['Res'][row], dihedrals['Angle'][row], 
               dihedrals['1'][row], dihedrals['2'][row],
               dihedrals['3'][row], dihedrals['4'][row],
               dihedrals['Res'][row] + '-' + dihedrals['Angle'][row])

ARG & $\phi$ & C & N & CA & C & \includegraphics[height=1in]{ARG-phi.png} \\
ARG & $\psi$ & N & CA & C & N & \includegraphics[height=1in]{ARG-psi.png} \\
ARG & $\omega$ & CA & C & N & CA & \includegraphics[height=1in]{ARG-omega.png} \\
ARG & $\chi_1$ & CG & CB & CA & N & \includegraphics[height=1in]{ARG-chi1.png} \\
ARG & $\chi_2$ & CD & CG & CB & CA & \includegraphics[height=1in]{ARG-chi2.png} \\
ARG & $\chi_3$ & NE & CD & CG & CB & \includegraphics[height=1in]{ARG-chi3.png} \\
ARG & $\chi_4$ & CZ & NE & CD & CG & \includegraphics[height=1in]{ARG-chi4.png} \\
ARG & $\chi_5$ & NH1 & CZ & NE & CD & \includegraphics[height=1in]{ARG-chi5.png} \\
HIS & $\phi$ & C & N & CA & C & \includegraphics[height=1in]{HIS-phi.png} \\
HIS & $\psi$ & N & CA & C & N & \includegraphics[height=1in]{HIS-psi.png} \\
HIS & $\omega$ & CA & C & N & CA & \includegraphics[height=1in]{HIS-omega.png} \\
HIS & $\chi_1$ & CG & CB & CA & N & \includegraphics[height=1in]{HIS-chi1.png} \\
HIS & $\chi_2$ & CD2 & CG & CB 